In [ ]:
1

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
df = pd.read_excel(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/culture/Conservatoire/Copie de Copie de Elèves 2022-23 - dominantes + montant facturé + tranche HA v2.xlsx",
)
df

In [ ]:
df["Indication géographique"].fillna("Hors EMS", inplace=True)
df["Enfant de la fratrie"].fillna(1, inplace=True)
df.Dominante.fillna(1, inplace=True)

In [ ]:
df.groupby("produit").Nom.count().sum()

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
from utils import StrasbourgSurveyScenario

In [ ]:
count = len(df)
sample_count = 1
individu_df = pd.DataFrame(
    {
        "famille_id": list(range(count * sample_count)),
    }
)
sample_ids = np.repeat(list(range(sample_count)), count)
indiv_ids = np.tile(list(range(count)), sample_count)
# sample_qfrule = np.tile(product_df.qfrule, sample_count)

agent = df["Indication géographique"].str.contains("Agent")
habitant = df["Indication géographique"].str.contains("Habitant")
famille_df = pd.DataFrame(
    {
        "sample_id": sample_ids,
        "strasbourg_conservatoire_base_ressources": 100000,
        "agent_ems": np.tile(agent, sample_count),
        "habitant_ems": np.tile(habitant, sample_count),
    }
)

menage_df = pd.DataFrame({})
foyerfiscaux_df = pd.DataFrame({})

individu_df["famille_role_index"] = 0
individu_df["foyer_fiscal_id"] = individu_df.famille_id
individu_df["foyer_fiscal_role_index"] = 0
individu_df["menage_id"] = individu_df.famille_id
individu_df["menage_role_index"] = 0

data = dict(
    input_data_frame_by_entity=dict(
        individu=individu_df,
        famille=famille_df,
        menage=menage_df,
        foyer_fiscal=foyerfiscaux_df,
    )
)

In [ ]:
scenario = StrasbourgSurveyScenario(base, data)

In [ ]:
v = scenario.simulation.calculate(
    "strasbourg_conservatoire_traditionnel_enf12", base_period
)

In [ ]:
pd.Series(v).unique()

In [ ]:
df[(~(df.MontantFactureSurEleve == v)) * (df.produit == "traditionnel_enf12")]

In [ ]:
(df.produit == "traditionnel_enf12").sum()